# Include 

In [2]:
import json
import numpy as np
import os
import random

from os.path import expanduser

# Definitions 

# Main course

# Dataset description

* source: https://www.geoapify.com/download-all-the-cities-towns-villages/
* file for russian locations: https://www.geoapify.com/data-share/localities/ru.zip

## Load geography

In [3]:
base = expanduser("~/database/shad/generate_text/ru")

In [4]:
filenames = ["city", "hamlet", "town", "village"]

In [5]:
dataset_raw = [json.loads(line) for current_name in filenames for line in open(os.path.join(base, f"place-{current_name}.ndjson"))]

In [6]:
len(dataset_raw)

253192

In [7]:
sum(1 for r in dataset_raw if "name" not in r)

57

In [8]:
names = [r["name"] for r in dataset_raw if "name" in r]

In [9]:
len(names)

253135

In [10]:
uniq_names = list(set(names))

In [11]:
len(uniq_names)

77411

In [12]:
random.shuffle(uniq_names)

In [13]:
uniq_names[:10]

['Большие Рожки',
 'Хорошие Воды',
 'Исанино',
 'Колташи',
 'Кулары',
 'Тургояк',
 'ДРСУ-1',
 'Шанай',
 'Таскаиха',
 'Воргониха']

In [14]:
alphabet=sorted(list(set(c for r in uniq_names for c in r)))

In [15]:
"".join(alphabet)

' "(),-./0123456789IVXc\xa0«»́ЁАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЫЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё№'

In [16]:
ss_alphabet = alphabet + ["<", ">"]

In [17]:
c2ind = {c:ind for ind, c in enumerate(ss_alphabet)}
ind2c = {ind:c for ind, c in enumerate(ss_alphabet)}

In [18]:
ss_names = ["<" + v + ">" for v in uniq_names]

In [19]:
ss_names[:10]

['<Большие Рожки>',
 '<Хорошие Воды>',
 '<Исанино>',
 '<Колташи>',
 '<Кулары>',
 '<Тургояк>',
 '<ДРСУ-1>',
 '<Шанай>',
 '<Таскаиха>',
 '<Воргониха>']

In [20]:
text = "".join(ss_names)

In [21]:
v_sz = len(set(ss_alphabet))
v_sz

93

In [23]:
text[:2000]

'<Большие Рожки><Хорошие Воды><Исанино><Колташи><Кулары><Тургояк><ДРСУ-1><Шанай><Таскаиха><Воргониха><Киска-Елга><Альмеева><Мордовский Бугуруслан><Кобляки><Мочаки-Первые><Парца><Широгорье><Талисман><Зюзинка><Обновление><Бачгины><Сардыган><Семановцы><Таратухино><Христофорово><Вельяминовка><Новоилецк><Балма><Ольховка><посёлок дома отдыха «Румянцево»><Нижняя Тига><Цветочный><Залоинка><Шиванда><Новые Ключищи><Тирякле><Малое Займище><Обидимо><Арметрахимово><Владимиров><Поляновка><Легчищево><Шагаки><Крашняково><Куртеково><Князевое><Развильное><Сасылыкан><Александро-Ерша><Бодалино><Новогригорьевская><Лакедемоновка><Торжнево><Корезино><Хыркасы><Усть-Есь><Гоголины><Рузские просторы><Маральский><Попелёво><Борисовец><Алманчино><Верховина><Дигилевка><Шемячкино><Верхнее Гаквари><Иляшкино><Немерь><Сегежа><Жестылево><Кара-Паго><Васькин Поток><Лучинка><Боборыкино><Ново-Кусково><Винищи><Надовражино><Атюловский><Борцино><Сандин 2-й><Новодворский><Колодезное><Замошанье><Верхние Сунары><Паршин><Шманы><Тай

In [38]:
def gen_pieces(text, seq_len):
    n = len(text)
    p = 0
    first = True
    while True:
        if p + seq_len > n:
            p = (p + 1) % seq_len
            first = True
        yield first, text[p:p + seq_len]
        first = False
        p += seq_len

In [39]:
it = gen_pieces("абвгдеёжзиклмн", 3)

In [40]:
[next(it) for _ in range(20)]

[(True, 'абв'),
 (False, 'где'),
 (False, 'ёжз'),
 (False, 'икл'),
 (True, 'бвг'),
 (False, 'деё'),
 (False, 'жзи'),
 (False, 'клм'),
 (True, 'вгд'),
 (False, 'еёж'),
 (False, 'зик'),
 (False, 'лмн'),
 (True, 'абв'),
 (False, 'где'),
 (False, 'ёжз'),
 (False, 'икл'),
 (True, 'бвг'),
 (False, 'деё'),
 (False, 'жзи'),
 (False, 'клм')]

# RNN

In [41]:
hsz = 100 # hidden size
vsz = len(ss_alphabet) # vocabulary size
seq_length = 25 # the size of window
lr = 1e-3

In [42]:
Wxh = np.random.randn(hsz, vsz) * 0.01
Whh = np.random.randn(hsz, hsz) * 0.01
Why = np.random.randn(vsz, hsz) * 0.01
bh = np.zeros((hsz, 1))
by = np.zeros((vsz, 1))

In [ ]:
def calcLoss(ind_input, ind_target, hprev):
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    for t in range(len(ind_input)):
        xs[t] = np.zeros((vsz, 1))
        xs[t][ind_input[t]] = 1
        hs[t] = np.tanh(Wxh@xs[t] + Whh@hs[t-1] + bh)
        ys[t] = Why@hs[t] + by
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
        loss -= np.log(ps[t][ind_target[t], 0])
    dWhh, dWxh, dWhy = np.zeros_like(Whh), np.zeros_like(Wxh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(ind_input))):
        dy = np.copy(ps[t])
        dy[ind_target[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += dy@hs[t].T
        dby += dy
        dh = Why.T @ dy + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += dhraw @ xs[t].T
        dWhh += dhraw @ hs[t-1].T
        dhnext = Whh.T @ dhraw
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[-1]